In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import os

import pickle

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

#Evalaluation
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

# Grid
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV

#Performance
%load_ext memory_profiler

D:\WindowsApps\Anaconda\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load

In [2]:
#os.chdir(r"D:/My Computer/")
#os.getcwd()

In [3]:
#Titanic
path = r"C:/Users/Nicol/Google Drive/Learning/Jupyter/Titanic"
train_df = pd.read_csv(open(os.path.join(path, "clean_train.csv"), "r")) 
test_df = pd.read_csv(open(os.path.join(path, "clean_test.csv"), "r")) 

X = train_df.drop(["Survived"] , axis=1)
y = train_df["Survived"]

oosample  = test_df.drop(["PassengerId"] , axis=1).copy()
print(X.shape, y.shape, oosample.shape)

results=[]
def save(model, modelname):
    model.fit(X, y)
    submission = model.predict(oosample)
    df = pd.DataFrame({'PassengerId':test_df.PassengerId, 
                           'Survived':submission})
    print(len(df))
    df.to_csv(("Titanic/submissions/{}.csv".format(modelname)),header=True,index=False)
    
    # CV and Save Scores
    trainingscore = (grid.best_score_*100)
    results.append([(trainingscore),("{}".format(modelname)), grid.best_estimator_])
    print(trainingscore)
    print(grid.best_params_)

    with open((r"Titanic/Pickle/{}.pickle".format(modelname)), 'wb') as f: pickle.dump(model, f)

(891, 8) (891,) (418, 8)


In [4]:
# Stratified Cross Validation    
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

In [5]:
# use train/test split with different random_state values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((668, 8), (668,), (223, 8), (223,))

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Pclass       891 non-null int64
Sex          891 non-null int64
Age          891 non-null int64
Fare         891 non-null int64
Embarked     891 non-null int64
Title        891 non-null float64
IsAlone      891 non-null int64
Age*Class    891 non-null int64
dtypes: float64(1), int64(7)
memory usage: 55.8 KB


In [7]:
# import seaborn as sns; sns.set()
# sns.pairplot(train_df, hue='Survived', size=1.5);

# Generative Classification
Probabilistically determine the label for a new point

## Gaussian

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

In [9]:
%%time
%%memit
model = GaussianNB()
model.fit(X_train, y_train)
y_pred= model.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred)*100)
print(cross_val_score(model, X, y, cv=10, scoring='accuracy').mean()*100)

74.8878923767
72.5175632732
74.8878923767
72.5175632732
74.8878923767
72.5175632732
74.8878923767
72.5175632732
peak memory: 131.20 MiB, increment: 0.47 MiB
Wall time: 2.01 s


## Logistic


In [10]:
%%time
%%memit
model= LogisticRegression()

#Fit Model

scores= cross_val_score(model, X, y, cv=10, scoring='accuracy')
print(scores.mean()*100)

80.4752298264
80.4752298264
80.4752298264
peak memory: 131.88 MiB, increment: 0.67 MiB
Wall time: 1.65 s


# Non-Parametric

# Esemble Method
## Bagging

https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
HyperParameters:
- max_features: limits number of number of features in a tree
- n_estimators: # of trees built before average prediciton is made
- min_sample_leaf: End node of trees. Too small = more noise. For Regression tree.
- n_jobs: computer processors utilized. -1 = no restrictions
- random_state: seed()

In [11]:
tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=300, max_samples=0.8,
                        random_state=1)

print(cross_val_score(bag, X, y, cv=10, scoring='accuracy').mean()*100)

81.0445749631


In [12]:
print(np.arange(0.5, 1, 0.1))
print([.5,.6,.7])

[ 0.5  0.6  0.7  0.8  0.9]
[0.5, 0.6, 0.7]


In [18]:
%%time
%%memit
param_grid ={'n_estimators': np.arange(20, 81, 10),
             'max_samples': np.arange(.01, .8, 0.05),
             'max_features': np.arange(0.3, .7,.1)}

tree = DecisionTreeClassifier()
#bag = BaggingClassifier(tree)

grid = GridSearchCV(BaggingClassifier(tree),
                    param_grid, cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "Bagger_ensemble")

Fitting 5 folds for each of 448 candidates, totalling 2240 fits
418
80.5586592179
{'max_features': 0.5, 'max_samples': 0.6100000000000001, 'n_estimators': 40}
peak memory: 115.52 MiB, increment: 3.32 MiB
Wall time: 1min 34s


[Parallel(n_jobs=1)]: Done 2240 out of 2240 | elapsed:  1.6min finished


In [ ]:
with open('Titanic/Pickle/Bagger_ensemble.pickle', 'rb') as f: Bagging_ensemble = pickle.load(f)
Bagging_ensemble.score(X, y)

## Random Forest

In [ ]:
%%time
%%memit
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

In [ ]:
RandomForestClassifier().get_params().keys()

In [ ]:
%%time
%%memit
param_grid ={'max_depth': np.arange(6, 11, 1),
             'n_estimators':np.arange(350, 450, 25),
             'max_features':np.arange(0.5,.81, 0.05),
            'max_leaf_nodes':np.arange(6, 10, .5)}
#param_grid ={'n_estimators':[200]}

from sklearn import feature_selection

#model = feature_selection.RFE(RandomForestClassifier())
model= RandomForestClassifier()

grid = GridSearchCV(model,
                    param_grid, cv=cv,
                    scoring='accuracy',
                    verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "Random_Forest")

## Gradient Boosting Classifier

GB builds an additive model in a forward stage-wise fashion; it allows for the optimization of arbitrary differentiable loss functions.

In [ ]:
#?GradientBoostingClassifier

In [ ]:
%%time
%%memit
param_grid ={'n_estimators':np.arange(100, 301, 25),
            'loss': ['deviance', 'exponential'],
            'learning_rate':np.arange(0.01, 0.32,.05),
            'max_depth': np.arange(2, 4.1, .5)}

grid = GridSearchCV(GradientBoostingClassifier(),
                    param_grid,cv=cv,
                    scoring='accuracy',
                    verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "Gradient_Boosting")

## AdaBoostClassifier

In [ ]:
#?AdaBoostClassifier

In [ ]:
%%time
%%memit
param_grid ={'n_estimators':np.arange(50, 301, 25),
            'learning_rate':np.arange(.1, 4, .5)}

grid = GridSearchCV(AdaBoostClassifier(),
                    param_grid,cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y);
save(grid.best_estimator_, "AdaBoost_Ensemble")

In [ ]:
## Add Stochastic Gradient Boosting

## KNN

In [ ]:
#?KNeighborsClassifier

In [ ]:
%%time
%%memit
param_grid ={'n_neighbors': np.arange(1,21,1),
            'weights':['uniform','distance']
            }

grid = GridSearchCV(KNeighborsClassifier(),
                    param_grid,cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "KNN")

# Discriminative Classification
Model new points by seeing where it falls upon a divide.
Fast prediction phase, work well in high dimensional data, versatile

Costly at high quantities of data

## Support Vector Classifier
Creates a linear divide between point to classify. Maximizes the distance of the discriminatory margin.

Hyperparameters:
- C: Hardness of the margin. Higher C, less softening.


Radial Basis Function (RBF)
- Gamma: how far the influence of a single training example raches. low=far, high=close, Inverse of the radius of influence of samples selected by the model as support vectors.

In [ ]:
%%time
%%memit

# Define Model
model = svm.LinearSVC()
#Fit Model
scores= cross_val_score(model, X, y, cv=10, scoring='accuracy')
print(scores.mean()*100)

#submit(svm.LinearSVC(), name="80linear_svc.csv")

In [ ]:
#?svm.SVC

In [ ]:
%%time
%%memit
param_grid = [
  {'C': np.arange(25, 101, 25), 'kernel': ['linear']},
  {'C': np.arange(25,176,50), 'gamma': np.logspace(-1, -4, 4), 'kernel': ['rbf']}
 ]

grid = GridSearchCV(svm.SVC(),
                    param_grid, cv=cv,
                    scoring='accuracy', verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "SVCdouble_kernel")

In [ ]:
%%time
%%memit
C_range = np.logspace(2, 4, 4)
gamma_range = np.logspace(-1, -4, 4)
param_grid = dict(gamma=gamma_range, C=C_range)

grid = GridSearchCV(svm.SVC(),
                    param_grid, cv=cv,
                    scoring='accuracy', verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "SVC2")

## PCA + SVC Pipeline

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=6, whiten=True, random_state=42, svd_solver='randomized')
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)
# pipeline!

param_grid = [{'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]},
              {'svc__C': [0.001, 0.01, 0.1, 1, 10, 50, 100, 150, 1000, 1500], 'svc__kernel': ['linear']}]
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(model, param_grid, cv=cv)

%time grid.fit(X, y)
save(grid.best_estimator_, "PCA_SVC")

In [ ]:
import csv
with open("Titanic/results.csv", 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(results)

In [ ]:
results = pd.read_csv("Titanic/results.csv").transpose()

In [ ]:
results

## MODEL ENSEMBLE

In [19]:
open_file = open("Titanic/Pickle/AdaBoost_Ensemble.pickle", "rb")
Ada_Boost_Ensemble = pickle.load(open_file)
open_file.close()

open_file = open("Titanic/Pickle/Bagger_ensemble.pickle", "rb")
Bagger_ensemble = pickle.load(open_file)
open_file.close()

open_file = open("Titanic/Pickle/Gradient_Boosting.pickle", "rb")
Gradient_Boosting = pickle.load(open_file)
open_file.close()

open_file = open("Titanic/Pickle/KNN.pickle", "rb")
KNN = pickle.load(open_file)
open_file.close()

open_file = open("Titanic/Pickle/PCA_SVC.pickle", "rb")
PCA_SVC = pickle.load(open_file)
open_file.close()

open_file = open("Titanic/Pickle/Random_Forest.pickle", "rb")
Random_Forest = pickle.load(open_file)
open_file.close()

open_file = open("Titanic/Pickle/SVCdouble_kernel.pickle", "rb")
SVCdouble_kernel = pickle.load(open_file)
open_file.close()

open_file = open("Titanic/Pickle/SVC2.pickle", "rb")
SVC2 = pickle.load(open_file)
open_file.close()

In [21]:
from sklearn.ensemble import VotingClassifier
?VotingClassifier

In [ ]:
ensemble = VotingClassifier(estimators=[
    ('ada',Ada_Boost_Ensemble),
    ('bagger',Bagger_ensemble),
    ('gradientboost',Gradient_Boosting),
    #('knn',KNN),
    #('pca_svc',PCA_SVC)
    ('svc1',SVCdouble_kernel),
    ('svc2',SVC2) 
], voting='soft')
                           # weights=[2,2,2,1,1,1,1])
                            #flatten_transform=True)
ensemble.fit(X,y)

In [22]:
est=[
    ('ada',Ada_Boost_Ensemble),
    ('bagger',Bagger_ensemble),
    ('gradientboost',Gradient_Boosting),
    ('knn',KNN),
    ('pca_svc',PCA_SVC),
    ('svc1',SVCdouble_kernel),
    ('svc2',SVC2)
]

ensemble = VotingClassifier(estimators=est,
                            #weights=[2,2],
                            #flatten_transform=True,
                            voting='soft')
ensemble.fit(X,y)

VotingClassifier(estimators=[('ada', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.6000000000000001, n_estimators=200,
          random_state=None)), ('bagger', BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
      ...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         n_jobs=1, voting='soft', weights=None)

In [ ]:
print(ensemble.predict_proba(X))

In [ ]:
ensemble.score(X,y)

In [ ]:
def ensembling(model, modelname):
    model.fit(X, y)
    submission = model.predict(oosample)
    df = pd.DataFrame({'PassengerId':test_df.PassengerId, 
                           'Survived':submission})
    print(len(df))
    df.to_csv(("Titanic/submissions/{}.csv".format(modelname)),header=True,index=False)

In [ ]:
ensembling(ensemble, "ensemble1")

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
y = np.array([1, 1, 1, 2, 2, 2])
eclf1 = VotingClassifier(estimators=[
    ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X, y)
print(eclf1.predict(X))
eclf2 = VotingClassifier(estimators=[
    ('lr', clf1), ('rf', clf2), ('gnb', clf3)],
                         voting='soft')
eclf2 = eclf2.fit(X, y)
print(eclf2.predict(X))
eclf3 = VotingClassifier(estimators=[
    ('lr', clf1), ('rf', clf2), ('gnb', clf3)],
                         voting='soft', weights=[2,1,1])
eclf3 = eclf3.fit(X, y)
print(eclf3.predict(X))